In [2]:
import subprocess
from pathlib import Path

import numpy as np
import nibabel as nib

from resolve_par import resolve_par

phatom_root_dir = Path.cwd().resolve().parent
xcat_dir = phatom_root_dir / "XCAT"
output_raw_root_dir = phatom_root_dir / "output_raw"
output_nii_root_dir = phatom_root_dir / "output_nii"
parameters_dir = phatom_root_dir / "parameters"

output_prefix = "color_code_test"
param_file_name = "color_code_test_512x512x205.par"

raw_dir = output_raw_root_dir / output_prefix
raw_dir.mkdir(parents=True, exist_ok=True)

nii_dir = output_nii_root_dir / output_prefix
nii_image_dir = nii_dir / "image"
nii_label_dir = nii_dir / "label"
nii_image_dir.mkdir(parents=True, exist_ok=True)
nii_label_dir.mkdir(parents=True, exist_ok=True)

xcat_dir = phatom_root_dir / "XCAT"
assert xcat_dir.exists()
param_path = parameters_dir / param_file_name
assert param_path.exists()

params = resolve_par(param_path)
pixel_width_cm = params["pixel_width"]
slice_width_cm = params["slice_width"]
array_size = int(params["array_size"])
startslice = params["startslice"]
endslice = params["endslice"]
depth = int(endslice - startslice + 1)
out_frames = int(params["out_frames"])

In [3]:
args = [
    "./dxcat2",
    str(param_path),
    str(raw_dir/output_prefix),
]
subprocess.run(args, cwd=xcat_dir)

  GetIntParm: parameter slice_output not found, returning default: 0
  GetIntParm: parameter mode4_include_tumor not found, returning default: 0
  GetIntParm: parameter output_material_id not found, returning default: 0
  GetIntParm: parameter use_activ_table not found, returning default: 0
  GetStrParm: parameter activ_table_filename not found, returning default: activ_table.txt
  GetDblParm: parameter prostate_transx not found, returning default: 0.0000e+00
  GetDblParm: parameter prostate_transy not found, returning default: 0.0000e+00
  GetDblParm: parameter prostate_transz not found, returning default: 0.0000e+00
0 parameters input but not used



License good for 280.210856 more days

Reading diaphragm_curve.dat motion file
Reading ap_curve.dat motion file
Setting Diaphragm Height
Setting Phantom Dimensions
Setting Bones
Setting Torso Dimensions
Setting Chest Muscle
Setting Arm and Leg Muscles
Setting Chest Skin Dimensions
Setting Breast Dimensions
Setting Head Dimensions
Setting Right Arm Dimensions
Setting Left Arm Dimensions
Setting Right Leg Dimensions
Setting Left Leg Dimensions
Setting Right Arm Skin Dimensions
Setting Left Arm Skin Dimensions
Setting Right Leg Skin Dimensions
Setting Left Leg Skin Dimensions
Phantom rotation
----Center of Rotation = 96.898720, 92.160019, 64.062500 mms
----Angles (x, y, z) = 0.000000 0.000000 0.000000 degrees

Setting inner layer to skull
Setting inner layer to skull
Setting inner layer to skull
Setting inner layer to skull
Setting inner layer to skull
Setting inner layer to skull
Setting inner layer to sternum
Setting inner layer to sacrum
Setting inner layer to rrib1
Setting inner laye

CompletedProcess(args=['./dxcat2', '/media/F/sj/Data/Phatom/parameters/color_code_test_512x512x205.par', '/media/F/sj/Data/Phatom/output_raw/color_code_test/color_code_test'], returncode=0)

In [4]:
pixel_width_mm = pixel_width_cm * 10
slice_width_mm = slice_width_cm * 10
spacing = (pixel_width_mm, pixel_width_mm, slice_width_mm) # (x, y, z)

for raw_file in raw_dir.glob("*_act_*.bin"):
    print(raw_file)
    with open(raw_file, "rb") as f:
        raw_data = np.fromfile(f, dtype=np.float32)
    expected_shape = (depth, array_size, array_size) # 原始轴顺序为(z, y, x)
    expected_size = np.prod(expected_shape)
    assert raw_data.size == expected_size
    data_3d = raw_data.reshape(expected_shape) # 原始轴顺序为(z, y, x)
    data_3d = data_3d.transpose(2, 1, 0) # 转换为(x, y, z)
    data_3d = np.flip(data_3d, axis=(0,1))
    data_3d = data_3d.astype(np.uint8)
    affine = np.eye(4)
    affine[(0,1,2), (0,1,2)] = spacing
    nii_img = nib.Nifti1Image(data_3d, affine)
    nib.save(nii_img, nii_label_dir / raw_file.name.replace(".bin", ".nii.gz"))


/media/F/sj/Data/Phatom/output_raw/color_code_test/color_code_test_act_1.bin


/media/F/sj/Data/Phatom/output_raw/color_code_test/color_code_test_act_av.bin


In [5]:
for raw_file in raw_dir.glob("*_atn_*.bin"):
    print(raw_file)
    with open(raw_file, "rb") as f:
        raw_data = np.fromfile(f, dtype=np.float32)
    expected_shape = (depth, array_size, array_size) # 原始轴顺序为(z, y, x)
    expected_size = np.prod(expected_shape)
    assert raw_data.size == expected_size
    data_3d = raw_data.reshape(expected_shape) # 原始轴顺序为(z, y, x)
    data_3d = data_3d.transpose(2, 1, 0) # 转换为(x, y, z)
    data_3d = np.flip(data_3d, axis=(0,1))
    affine = np.eye(4)
    affine[(0,1,2), (0,1,2)] = spacing
    nii_img = nib.Nifti1Image(data_3d, affine)
    nib.save(nii_img, nii_image_dir / raw_file.name.replace(".bin", ".nii.gz"))

/media/F/sj/Data/Phatom/output_raw/color_code_test/color_code_test_atn_av.bin
/media/F/sj/Data/Phatom/output_raw/color_code_test/color_code_test_atn_1.bin
